In [1]:
!apt-get install openjdk-11-jdk-headless -qq
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar -xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"

import findspark
findspark.init()
from pyspark.sql import SparkSession
# spark = SparkSession.builder.master("local[*]").getOrCreate()
# sc = spark.sparkContext

In [2]:
spark = SparkSession.builder.appName("Python Spark SQL Hive").enableHiveSupport().getOrCreate()

In [3]:
spark.sql("CREATE TABLE IF NOT EXISTS src (key INT, value STRING) USING hive")
spark.sql("show tables").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|      src|      false|
+---------+---------+-----------+



In [4]:
spark.sql("SELECT * FROM src").show()

+---+-----+
|key|value|
+---+-----+
+---+-----+



In [5]:
spark.sql('INSERT INTO src (key, value) VALUES (238, "val_238")')
spark.sql('INSERT INTO src (key, value) VALUES (126, "val_126")')
spark.sql('INSERT INTO src (key, value) VALUES (235, "val_235")')

DataFrame[]

In [6]:
spark.sql("SELECT * FROM src").show()

+---+-------+
|key|  value|
+---+-------+
|238|val_238|
|235|val_235|
|126|val_126|
+---+-------+



In [ ]:
spark.sql("SELECT COUNT(*) FROM src").show()

+--------+
|count(1)|
+--------+
|       3|
+--------+



In [ ]:
sqlDF = spark.sql("SELECT key, value FROM src WHERE key < 200 ORDER BY key")
sqlDF.show()

+---+-------+
|key|  value|
+---+-------+
|126|val_126|
+---+-------+



In [ ]:
stringsDS = sqlDF.rdd.map(lambda row: "Key: %d, Value: %s" % (row.key, row.value))
for record in stringsDS.collect():
    print(record)

Key: 126, Value: val_126


In [ ]:
from pyspark.sql import Row
Record = Row("key", "value")
recordsDF = spark.createDataFrame([Record(i, "val_" + str(i)) for i in range(1, 101)])
recordsDF.createOrReplaceTempView("records")

recordsDF.show()

+---+------+
|key| value|
+---+------+
|  1| val_1|
|  2| val_2|
|  3| val_3|
|  4| val_4|
|  5| val_5|
|  6| val_6|
|  7| val_7|
|  8| val_8|
|  9| val_9|
| 10|val_10|
| 11|val_11|
| 12|val_12|
| 13|val_13|
| 14|val_14|
| 15|val_15|
| 16|val_16|
| 17|val_17|
| 18|val_18|
| 19|val_19|
| 20|val_20|
+---+------+
only showing top 20 rows



In [ ]:
spark.sql("show tables").show()
spark.sql("SELECT * FROM records").show()

+---------+---------+-----------+
|namespace|tableName|isTemporary|
+---------+---------+-----------+
|  default|      src|      false|
|         |  records|       true|
+---------+---------+-----------+

+---+------+
|key| value|
+---+------+
|  1| val_1|
|  2| val_2|
|  3| val_3|
|  4| val_4|
|  5| val_5|
|  6| val_6|
|  7| val_7|
|  8| val_8|
|  9| val_9|
| 10|val_10|
| 11|val_11|
| 12|val_12|
| 13|val_13|
| 14|val_14|
| 15|val_15|
| 16|val_16|
| 17|val_17|
| 18|val_18|
| 19|val_19|
| 20|val_20|
+---+------+
only showing top 20 rows



In [ ]:
spark.sql('INSERT INTO src (key, value) VALUES (12, "val_12")')
spark.sql('INSERT INTO src (key, value) VALUES (23, "val_23")')
spark.sql('INSERT INTO src (key, value) VALUES (75, "val_75")')
spark.sql('INSERT INTO src (key, value) VALUES (25, "val_25")')

DataFrame[]

In [ ]:
spark.sql("SELECT * FROM records r JOIN src s ON r.key = s.key").show()

+---+------+---+------+
|key| value|key| value|
+---+------+---+------+
| 12|val_12| 12|val_12|
| 23|val_23| 23|val_23|
| 25|val_25| 25|val_25|
| 75|val_75| 75|val_75|
+---+------+---+------+

